In [1]:
import numpy as np
from functools import partial
import ipywidgets as widgets
from IPython.display import display

In [2]:
from pydrake.geometry.optimization import IrisOptions, HPolyhedron, Hyperellipsoid
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions
from pydrake.all import (PiecewisePolynomial, 
                         InverseKinematics, 
                         Sphere, 
                         Rgba, 
                         RigidTransform, 
                         RotationMatrix, 
                         Solve,
                         MathematicalProgram,
                         RollPitchYaw,
                         Cylinder,
                         VPolytope,
                         Role)
import time
import pydrake

In [3]:
from environments import get_environment_builder

plant_builder = get_environment_builder('2DOFFLIPPER')
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(usemeshcat=True)

scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7000


http://localhost:7000


In [4]:
inspector = scene_graph.model_inspector()
b = plant.GetBodyFrameIdIfExists(plant.GetBodyByName("roi_box", plant.GetModelInstanceByName("roi_box")).index(), )
ids = inspector.GetGeometries(b, Role.kIllustration)
vp = [VPolytope(scene_graph.get_query_output_port().Eval(scene_graph_context), id) for id in ids]
roi = HPolyhedron(vp[0])

In [5]:
from visibility_utils import (sample_in_union_of_polytopes, 
                              get_AABB_cvxhull,
                              )
from visualization_utils import plot_points
tmin, tmax, aabb = get_AABB_cvxhull([roi])
# pts = sample_in_union_of_polytopes(100, [roi], [tmin, tmax])
# pts[:,0] = 0
# plot_points(meshcat, pts, 'roitarg', size = 0.01)

/home/peter/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.17.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:

_offset_meshcat_2 = np.array([-1,-5, 1.5])
def show_pose(qvis, plant, plant_context, diagram, diagram_context, endeff_frame, show_body_frame = None):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    tf =plant.EvalBodyPoseInWorld(plant_context,  plant.GetBodyByName(endeff_frame))
    transl = tf.translation()+tf.rotation()@np.array([0,0.0,0.05])
    meshcat.SetObject(f"/drake/q",
                                   Sphere(0.1),
                                   Rgba(0,1,0,1))
    p = np.zeros(3)
    p[1] = qvis[0]
    p[2] = qvis[1]
    meshcat.SetTransform(f"/drake/q",RigidTransform(p+_offset_meshcat_2))
    
    
    if show_body_frame is not None:
        show_body_frame(RigidTransform(tf.rotation(), transl))

def show_ik_target(pose, meshcat, name):
    h = 0.2
    if 'targ' in name:
        colors = [Rgba(1,0.5,0, 0.5), Rgba(0.5,1,0, 0.5), Rgba(0.0,0.5,1, 0.5)]
    else:
        colors = [Rgba(1,0,0, 1), Rgba(0.,1,0, 1), Rgba(0.0,0.0,1, 1)]

    rot = pose.rotation()@RotationMatrix.MakeYRotation(np.pi/2)
    pos= pose.translation() +pose.rotation()@np.array([h/2, 0,0])
    meshcat.SetObject(f"/drake/ik_target{name}/triad1",
                                   Cylinder(0.01,0.2),
                                   colors[0])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad1",RigidTransform(rot, pos))
    rot = pose.rotation()@RotationMatrix.MakeXRotation(-np.pi/2)
    pos= pose.translation() +pose.rotation()@np.array([0,h/2,0])

    meshcat.SetObject(f"/drake/ik_target{name}/triad2",
                                   Cylinder(0.01,0.2),
                                   colors[1])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad2",RigidTransform(rot, pos))
    pos= pose.translation().copy()
    rot = pose.rotation()
    pos = pos + rot@np.array([0,0,h/2])
    meshcat.SetObject(f"/drake/ik_target{name}/triad3",
                                   Cylinder(0.01,0.2),
                                   colors[2])
    meshcat.SetTransform(f"/drake/ik_target{name}/triad3",RigidTransform(rot, pos))
show_body_frame = partial(show_ik_target, 
                          meshcat=meshcat, 
                          name='endeff_acutal', 
                          )
showres = partial(show_pose, 
                  plant = plant, 
                  plant_context = plant_context, 
                  diagram = diagram, 
                  diagram_context = diagram_context,
                  endeff_frame = 'iiwa_twoDOF_link_7',
                  show_body_frame=show_body_frame)

In [7]:
q_meshcat = np.zeros(plant.num_positions())

sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, description=f"q{i}"))

def handle_slider_change(change, idx):
    q_meshcat[idx] = change['new']
    showres(q_meshcat)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

In [8]:
#ik sliders 
from task_space_seeding_utils import solve_ik_problem, task_space_sampler_mp

show_ik_targets = partial(show_ik_target, 
                          meshcat=meshcat, 
                          name='endeff_targ', 
                          )

ik_solver = partial(solve_ik_problem, 
                    plant_ik = plant,
                    plant_context_ik = plant_context,
                    frames = [plant.GetFrameByName('iiwa_twoDOF_link_7')],
                    offsets = [np.array([0,0.0,0.05])],
                    collision_free=False,
                    track_orientation=False)
sliders_ik = []
pos_min = np.array([-1,-1])
pos_max = np.array([1.0,1])
names = ['y', 'z']
q0 = np.zeros(2)#np.array([ 0.23294, -0.2944 , -0.36706, -1.5944 , -0.16706,  0.     ,
       # 1.64567])
_ik_positions = np.zeros(2)

for i in range(2):
    sliders_ik.append(widgets.FloatSlider(min=pos_min[i], 
                                       max=pos_max[i], 
                                       value=0.5*(pos_max[i]-pos_min[i])+pos_min[i], 
                                       description=names[i]))
q_ik = np.zeros(plant.num_positions())
def handle_slider_change_ik(change, idx):
    _ik_positions[idx] = change['new']
    rot = RotationMatrix()
    pos = np.zeros(3)
    pos[1] = _ik_positions[0]
    pos[2] = _ik_positions[1]
    tf = RigidTransform(rot, pos)
    show_ik_targets(tf)
    res = ik_solver([tf], q0)
    q_ik[:] = res
    if res is not None:
        showres(res)

idx = 0
for slider in sliders_ik:
    slider.observe(partial(handle_slider_change_ik, idx = idx), names='value')
    idx+=1

for slider in sliders_ik:
    display(slider)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='z', max=1.0, min=-1.0)

In [9]:
#sample in roi using IK
starting_confs = [np.array([1.5,1.0]),
                  np.array([-0.3,-1.0])]
 
ts_samplers = []
for q0 in starting_confs:    
    plant.SetPositions(plant_context, q0)
    plant.ForcedPublish(plant_context)
    showres(q0)
    t0 = plant.EvalBodyPoseInWorld(plant_context,  plant.GetBodyByName("iiwa_twoDOF_link_7")).translation()  
    sample_handle_ts = partial(task_space_sampler_mp,
                            q0 = q0,
                            t0 = t0,
                            plant_builder = plant_builder,
                            frame_names = ['iiwa_twoDOF_link_7'],
                            offsets = [np.array([0,0.0,0.05])],
                            cvx_hulls_of_ROI = [roi],
                            ts_min = tmin,
                            ts_max = tmax,
                            collision_free = True,
                            track_orientation = False,
                            
                            )
    ts_samplers.append(sample_handle_ts)


In [14]:
q_obj = []
t_obj = []
Npts = 100
for sh in ts_samplers:
    q, t, _, res = sh(Npts,[])
    q_obj +=[q]
    t_obj +=[t]

for i,t in enumerate(t_obj):
    plot_points(meshcat, t, f"iksol{i}", size = 0.01)

100%|██████████| 5/5 [00:00<00:00, 126.91it/s]

100%|██████████| 5/5 [00:00<00:00, 119.92it/s]

100%|██████████| 5/5 [00:00<00:00, 223.10it/s]


100%|██████████| 5/5 [00:00<00:00, 171.66it/s]

100%|██████████| 5/5 [00:00<00:00, 99.45it/s]

100%|██████████| 5/5 [00:00<00:00, 78.09it/s]


In [11]:
# import time
# q_tot = np.concatenate(tuple(q_obj))
# for q in q_tot:
#     showres(q)
#     time.sleep(0.1)

In [12]:
import pickle, os, mcubes
from pydrake.all import SurfaceTriangle, TriangleSurfaceMesh
from visibility_utils import get_col_func
q_min = np.array([0, -1.8, -2.3])
q_max = np.array([0, 2.3,  2.3])
col_func_handle_ = get_col_func(plant, plant_context)
def check_collision_by_ik(q0,q1,q2, min_dist=1e-5):
    q = np.array([q1,q2])
    if np.any(q>q_max[1:]):
        return 0
    if np.any(q<q_min[1:]):
        return 0
    return 1.*col_func_handle_(q) 

def plot_collision_constraint(N = 50, q_min = q_min, q_max= q_max):
    if f"col_cons2d2{N}.pkl" in os.listdir('tmp'):
        with open(f"tmp/col_cons2d{N}.pkl", 'rb') as f:
            d = pickle.load(f)
            vertices = d['vertices']
            triangles = d['triangles']
    else:  
        vertices, triangles = mcubes.marching_cubes_func(
        tuple(
                q_min-0.5), tuple(
                q_max+0.5), 2, N, N, check_collision_by_ik, 0.5)
        with open(f"tmp/col_cons{N}.pkl", 'wb') as f:
                d = {'vertices': vertices, 'triangles': triangles}
                pickle.dump(d, f)

    tri_drake = [SurfaceTriangle(*t) for t in triangles]

    vertices += _offset_meshcat_2.reshape(-1,3)
    meshcat.SetObject("/collision_constraint/c1",
                                    TriangleSurfaceMesh(tri_drake, vertices),
                                    Rgba(0, 0.6, 0, 0.2))
    meshcat.SetObject("/collision_constraint/c2",
                                    TriangleSurfaceMesh(tri_drake, vertices),
                                    Rgba(0.6, 0.0, 0, 1), wireframe = True)
    

plot_collision_constraint(111)

In [13]:
for i,q in enumerate(q_obj):
    qext = np.concatenate((np.zeros((q.shape[0],1)),q), axis =1)
    plot_points(meshcat, qext+_offset_meshcat_2, f"ik _{i}", size = 0.03)

In [35]:
q.shape

(100, 2)